# Analysing feedback forms (Excel) from customers

Notebook to analyze a bunch of feedback forms recieved from customer. The forms are in excel format and a screenshot is given below. The data in them are read in using pandas and analyzed. One extra field was created to represent the department for which the team belongs. It was extracted from the filename of the feedback form.

In [ ]:
import os

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 16, 6

All feedback forms are kept in a folder

In [ ]:
basepath = "C:/Users/yy53393/Documents/feedbacks"

In [ ]:
forms = os.listdir(basepath)
#forms = [x for x in os.listdir(basepath) if x.endswith('xlsx') and x.startswith('feedback')]

In [ ]:
dfs = []

## Form format

![formformat](form_format.JPG)

Reading in all the forms and joining them to create a single dataframe.

In [ ]:
for form in forms:
    df = pd.read_excel(
        os.path.join(basepath,form), 
        skiprows=6, 
        usecols=[3,5,6], 
        nrows=6).iloc[:5]    
    df["Filename"] = form
    dfs.append(df)

In [ ]:
feedbacks = pd.concat(dfs)

In [ ]:
feedbacks = feedbacks.reset_index(drop=True)

In [ ]:
feedbacks

## Getting the department name from the filename

In [ ]:
def set_dept(filename):
    for dept in ['AS', 'CS', 'HS', 'SPE', 'ZDM', 'Driveline']:
        if dept in filename:
            return dept

In [ ]:
feedbacks['Department'] = feedbacks.Filename.apply(set_dept)

In [ ]:
feedbacks

## Number of feedback forms recieved

In [ ]:
len(feedbacks.Filename.value_counts())

## Feedback forms per department

In [ ]:
feedbacks.groupby(["Department"]).Filename.unique().apply(len).plot(kind='bar')

## Score details

In [ ]:
feedbacks.describe()

In [ ]:
feedbacks.groupby("Attribute").Score.agg([np.max, np.min, np.average, np.std])

## Feedback box plot grouped by attribute

In [ ]:
feedbacks.boxplot('Score', by='Attribute', )

## Department wise

In [ ]:
feedbacks.groupby("Department").Score.agg([np.max, np.min, np.average, len])

In [ ]:
feedbacks.groupby("Department").Score.agg([np.max, np.min, np.average, np.std]).plot(kind='bar')

## Attribute score per department

In [ ]:
for group, df in feedbacks.groupby(['Department']):
    ax = df.groupby('Attribute').Score.agg([np.max, np.min, np.average, np.std]).plot(kind='bar')
    ax.set_title("{} - {} feedbacks".format(group, df.shape[0]/5))

## Low feedback cases

In [ ]:
low = feedbacks[(feedbacks.Score < 7.5)]

In [ ]:
for group, df in low.groupby(['Department']):
    ax = df.groupby('Attribute').Score.agg([np.max, np.min, np.average, np.std, len]).plot(kind='bar')
    ax.set_title("{} - {} items".format(group, df.shape[0]))

## Comments word cloud

In [ ]:
from wordcloud import WordCloud

### Overall

In [ ]:
text = " ".join(feedbacks.Comments.dropna().values)
# Generate a word cloud image
wordcloud = WordCloud().generate(text)
import matplotlib.pyplot as plt
# lower max_font_size
wordcloud = WordCloud(width=800, height=450).generate(text)
plt.figure(figsize=(16,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

### Low

In [ ]:
text = " ".join(low.Comments.dropna().values)
# Generate a word cloud image
wordcloud = WordCloud().generate(text)
import matplotlib.pyplot as plt
# lower max_font_size
wordcloud = WordCloud(width=800, height=450).generate(text)
plt.figure(figsize=(16,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()